Importing the necessary libraries

In [18]:
import numpy as np
import pandas as pd
from numpy.linalg import norm

Functions

In [19]:
def standardize(x):
    if x.size == 0:
        raise ValueError("Input array x is empty.")
    mean = np.mean(x, axis=0)
    std_dev = np.std(x, axis=0)
    if np.any(std_dev == 0):
        raise ValueError("Standard deviation is zero for one or more features, cannot standardize.")
    x_std = (x - mean) / std_dev
    return x_std


Covariance

In [20]:
def covariance_matrix(x):
    if x.size == 0:
        raise ValueError("Input array x is empty.")
    m = x.shape[0]
    return (1 / (m - 1)) * x.T @ x


### QR Decomposition using Givens Rotation function

In [21]:
def givens_rotation(a, b):
    """Compute the cos and sin for Givens rotation."""
    if b == 0:
        return 1, 0
    else:
        r = np.hypot(a, b)
        c = a / r
        s = -b / r
        return c, s

In [22]:
def qr_decomposition_givens(A, tol=1e-8):
    """
    QR decomposition using Givens Rotations.
    Returns the matrices Q and R.
    """
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy()

    for i in range(n):
        for j in range(i + 1, m):
            if abs(R[j, i]) < tol:
                continue
            c, s = givens_rotation(R[i, i], R[j, i])
            G = np.eye(m)
            G[i, i], G[i, j], G[j, i], G[j, j] = c, -s, s, c

            R = G @ R
            Q = Q @ G.T

    return Q, R

In [23]:
def eigen_decomp(A, max_iter=100, tol=1e-8):
    """
    Compute the eigenvalues and eigenvectors using the QR algorithm with Givens rotations.
    """
    n = A.shape[0]
    A_k = A.copy()
    Q_total = np.eye(n)
    
    for _ in range(max_iter):
        Q, R = qr_decomposition_givens(A_k)
        A_k = R @ Q
        Q_total = Q_total @ Q
        
        if np.allclose(A_k - np.diag(np.diagonal(A_k)), 0, atol=tol):
            break

    eigenvalues = np.diagonal(A_k)
    eigenvectors = Q_total

    for i in range(n):
        eigenvectors[:, i] /= norm(eigenvectors[:, i])

    return eigenvalues, eigenvectors


In [24]:
# do PCA
def pca(x, threshold):
    x_std = standardize(x)
    cov_matrix = covariance_matrix(x_std)
    eigenvalues, eigenvectors = eigen_decomp(cov_matrix)
    total_variance = np.sum(eigenvalues)
    variance_ratio = eigenvalues / total_variance
    cumulative_variance_ratio = np.cumsum(variance_ratio)
    print(f"Explained variance ratio: {cumulative_variance_ratio}")
    n_components = np.argmax(cumulative_variance_ratio >= threshold) + 1
    V_k  = eigenvectors[:, :n_components]
    Z = np.dot(X_std, V_k)

    return Z, V_k, n_components

In [ ]:
# Normal Equation
def normal_equation(x, y):
    if x.size == 0 or y.size == 0:
        raise ValueError("Input arrays x and y are empty.")
    x = np.c_[np.ones(x.shape[0]), x]
    return np.linalg.inv(x.T @ x) @ x.T @ y